https://stanford.edu/class/msande448/2018/Final/Reports/gr2.pdf

### Imports

In [148]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
from datetime import datetime
from sklearn.linear_model import LinearRegression
from arch import arch_model
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# Configure Polars
cfg = pl.Config()
cfg.set_tbl_rows(50)

polars.config.Config

### Functions

In [10]:
def resample_stock_bars(
    df: pl.DataFrame,
    freq: str,
    timestamp_col: str = "timestamp",
    symbol_col: str = "symbol",
    volatility_window: int = 2,
    market_hours_only: bool = True,
    timezone: str = "UTC",
) -> pl.DataFrame:
    """
    Resample stock bar data from minute/hourly to a specified frequency using Polars.

    Parameters:
    -----------
    df : pl.DataFrame
        DataFrame with stock bar data (must have timestamp and symbol columns)
    freq : str
        Target frequency for resampling. Examples:
        - '6h' for 6 hours
        - '1d' or '24h' for daily
        - '4h' for 4 hours
        - '1w' for weekly
    timestamp_col : str
        Name of the timestamp column (default: "timestamp")
    symbol_col : str
        Name of the symbol column (default: "symbol")
    volatility_window : int
        Frequency to calculate the volatility in days
    market_hours_only : bool
        If True, only use market hours data for resampling (default: True)
    timezone : str
        Target timezone for the output data (default: 'UTC')

    Returns:
    --------
    pl.DataFrame
        Resampled DataFrame with the same structure as input
    """

    # Ensure timestamp is datetime and convert timezone
    df = df.with_columns([
        pl.col(timestamp_col).dt.convert_time_zone(timezone)
    ])

    # Calculate volatility window based on frequency
    def get_volatility_window(freq_str: str, vol_window_days: int) -> int:
        """Convert volatility window from days to number of periods based on frequency"""
        match = re.match(r"(\d+)([A-Za-z]+)", freq_str)
        if not match:
            return vol_window_days

        num = int(match.group(1))
        unit = match.group(2).lower()

        if unit == "d":
            return vol_window_days
        elif unit == "h":
            hours_per_day = 6.5
            total_hours = vol_window_days * hours_per_day
            return int(total_hours / num)
        elif unit == "w":
            return max(1, int(vol_window_days / 7))
        elif unit in ["m", "min"]:
            minutes_per_day = 6.5 * 60
            total_minutes = vol_window_days * minutes_per_day
            return int(total_minutes / num)
        else:
            return vol_window_days

    vol_window_periods = get_volatility_window(freq, volatility_window)

    # Filter for market hours if requested
    if market_hours_only:
        df = df.filter(
            (pl.col(timestamp_col).dt.hour() >= 9) &
            (pl.col(timestamp_col).dt.hour() < 16) &  # Market closes at 4pm
            (pl.col(timestamp_col).dt.weekday() < 5)  # Monday=0, Friday=4
        )

    # Resample using group_by_dynamic per symbol
    resampled = (
        df
        .sort([symbol_col, timestamp_col])
        .group_by(symbol_col)
        .agg([
            pl.col(timestamp_col)
            .dt.truncate(freq)
            .alias(timestamp_col),
            pl.col("open"),
            pl.col("high"),
            pl.col("low"),
            pl.col("close"),
            pl.col("volume"),
            pl.col("trade_count"),
            pl.col("vwap"),
        ])
        .explode([timestamp_col, "open", "high", "low", "close", "volume", "trade_count", "vwap"])
        .group_by([symbol_col, timestamp_col])
        .agg([
            pl.col("open").first(),
            pl.col("high").max(),
            pl.col("low").min(),
            pl.col("close").last(),
            pl.col("volume").sum(),
            pl.col("trade_count").sum(),
            pl.col("vwap").mean(),
        ])
    )

    # Calculate returns and volatility per symbol
    resampled = (
        resampled
        .sort([symbol_col, timestamp_col])
    #     .with_columns([
    #         # Calculate returns per symbol
    #         (pl.col("close").pct_change().over(symbol_col)).alias("returns"),
    #     ])
    #     .with_columns([
    #         # Calculate moving average per symbol
    #         (
    #             pl.col("close")
    #             .rolling_mean(window_size=vol_window_periods)
    #             .over(symbol_col)
    #         ).alias("moving_average")
    #     ])
    #     .with_columns([
    #         # Calculate volatility per symbol
    #         (
    #             pl.col("moving_average")
    #             .rolling_std(window_size=vol_window_periods)
    #             .over(symbol_col)
    #         ).alias("volatility")
    #     ])
    #     .sort([timestamp_col, symbol_col])
    )

    return resampled

### Read Data

In [155]:
data_path_raw = (
    Path.cwd()
    / "data/external"
    / "bars_data_20190106_to_20251219__20251224.parquet"
)

bars = pl.read_parquet(data_path_raw)

col_rename = {
    "close": "Close",
    "open": "Open",
    "high": "High",
    "low": "Low",
    # "returns": "Returns",
    # "volatility": "Volatility",
    "volume": "Volume",
    "vwap": "VWAP"
}

df_bars = resample_stock_bars(
    bars,
    timestamp_col="timestamp",
    symbol_col="symbol",
    freq="1d",
    volatility_window=10,    # In Days
    market_hours_only=True,
    timezone='America/New_York',
).rename(col_rename)

df_whole = df_bars.clone()
df_first_half = df_bars.filter(pl.col("timestamp") < pl.lit("2023-01-01").str.to_date())
df_second_half = df_bars.filter(pl.col("timestamp") > pl.lit("2023-01-01").str.to_date())

df_whole_pandas = df_bars.to_pandas().set_index("timestamp")
df_first_half_pandas = df_first_half.to_pandas().set_index("timestamp")
df_second_half_pandas = df_second_half.to_pandas().set_index("timestamp")

### GARCH Model

Formulation: $\sigma^2_t = \omega + \sum_{i=1}^p \alpha_i \epsilon^2_{t-i} + \sum^q_{j=1} \beta_j \sigma^2_{t-j}$

In [157]:
# calculate the log returns: log( P(t+1)/P(t) ), residuals: log_returns - mean,
# volatility: std wth 200 day rolling window

df_transformed = (
    df_whole.with_columns(
        (pl.col("Close").shift(-1) / pl.col("Close"))
        .log()
        .over("symbol")
        .alias("log_returns")
    )
    .with_columns(
        pl.col("log_returns")
        .rolling_std(200)
        .over("symbol")
        .alias("volatility")
    )
    .sort(["timestamp", "symbol"])
)
df_transformed.head(20)

symbol,timestamp,Open,High,Low,Close,Volume,trade_count,VWAP,log_returns,volatility
str,"datetime[ns, America/New_York]",f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BKR""",2019-01-07 00:00:00 EST,17.95,18.63,17.86,18.46,3.945225e6,31266.0,18.411853,0.028833,null
"""COP""",2019-01-07 00:00:00 EST,50.32,50.49,49.25,49.88,5.593103e6,52415.0,49.996752,0.012947,null
"""CTRA""",2019-01-07 00:00:00 EST,17.38,18.16,17.11,17.86,8.704615e6,60948.0,17.914162,0.013348,null
"""CVX""",2019-01-07 00:00:00 EST,81.97,83.03,80.81,82.55,4.691504e6,50358.0,82.402766,-0.004371,null
"""DVN""",2019-01-07 00:00:00 EST,17.77,18.51,17.5,18.23,6.631284e6,48309.0,18.202379,0.00765,null
"""EOG""",2019-01-07 00:00:00 EST,71.11,72.56,69.4,71.15,3.933068e6,44612.0,71.514462,0.010347,null
"""EQT""",2019-01-07 00:00:00 EST,17.75,19.12,17.64,18.31,5.97553e6,39713.0,18.513026,0.030126,null
"""EXE""",2019-01-07 00:00:00 EST,370.58,391.44,360.96,380.21,181880.0,54959.0,381.32066,0.03115,null
"""FANG""",2019-01-07 00:00:00 EST,78.27,80.31,77.29,79.55,1.963001e6,24764.0,79.251615,0.021637,null


$\sigma^2_t = \omega + \sum_{i=1}^p \alpha_i \epsilon^2_{t-i} + \sum^q_{j=1} \beta_j \sigma^2_{t-j}$

In [ ]:

# 1. Define a robust function to fit the model and return a Struct
def fit_garch(series: pl.Series) -> pl.Series:
    # Convert Polars series to numpy for the arch library
    if series.has_nulls():
        raise ValueError("Series contains null values!")

    # Safety check: GARCH needs enough data points
    if len(series) < 20:
        raise ValueError("Not enough length for GARCH model evaluation!")

    y = series.to_numpy()

    # Fit the model once
    model = arch_model(y, vol="GARCH", p=1, q=1)
    res = model.fit(disp="off")

    # Return both metrics packed in a DataFrame converted to a Struct
    struct_ = pl.DataFrame(
        {"cond_vol": res.conditional_volatility, "resid": res.resid}
    ).to_struct()
    return struct_



# 2. Apply the transformation
z_matrix = (
    df_transformed.drop_nulls(subset="log_returns").remove(pl.col("symbol")=="XLE")
    .with_columns(
        (pl.col("log_returns") * 100)
        .map_batches(
            fit_garch,
            return_dtype=pl.Struct({"cond_vol": pl.Float64, "resid": pl.Float64}),
        )
        .over("symbol")
        .alias("garch_results")
    )
    # Unpack the Struct into separate columns and calculate std_resid
    .unnest("garch_results")
    .with_columns((pl.col("resid") / pl.col("cond_vol")).alias("std_resid"))
).pivot(on="symbol", values="std_resid", index="timestamp")
z_matrix.head(20)

timestamp,BKR,COP,CTRA,CVX,DVN,EOG,EQT,EXE,FANG,HAL,KMI,MPC,OKE,OXY,PSX,SLB,TPL,TRGP,VLO,WMB,XOM
"datetime[ns, America/New_York]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2019-01-07 00:00:00 EST,1.46746,0.678797,0.913772,-0.398445,0.334764,0.547699,0.990775,0.717612,1.031928,0.61035,2.361736,0.475412,1.298287,0.559805,-0.143598,0.584623,0.43627,1.520157,0.155482,2.217893,0.714156
2019-01-08 00:00:00 EST,-0.115862,1.772172,-1.596814,1.022882,0.499225,2.163574,-0.659467,2.816962,0.813381,1.318387,0.264436,1.130239,0.749528,0.89799,0.539583,0.686932,-0.795587,0.808721,-0.098731,0.632475,0.485881
2019-01-09 00:00:00 EST,0.3471,1.117908,0.72577,0.057124,0.847345,-0.099859,-0.291439,-0.184639,-0.38286,1.302683,0.380108,-0.571606,0.238729,0.380978,0.791734,0.464979,0.229901,-0.899655,0.019336,-0.106931,-0.676124
2019-01-10 00:00:00 EST,-0.279298,-1.776218,1.459264,-1.315683,-0.296426,-0.354419,1.664737,0.056988,-0.87091,-0.807981,-0.553553,0.972752,-0.87398,-0.668207,-0.133993,-0.447717,-0.238081,-0.892506,1.098331,0.360539,-0.694803
2019-01-14 00:00:00 EST,-0.530134,0.275298,0.302083,0.273375,0.453266,0.025132,0.204679,0.757352,0.146198,-0.059664,0.231507,-0.493255,1.535972,-0.162084,-0.391249,0.459503,-0.243537,0.426918,-0.192902,0.331139,-0.008503
2019-01-15 00:00:00 EST,0.229688,-0.264267,0.640302,-0.528815,-1.057944,0.171343,-0.699224,0.057378,0.226409,-0.061292,0.531097,-0.087123,0.095043,-0.514886,0.402566,-0.660725,0.994066,0.187476,0.104838,1.090232,-0.073088
2019-01-16 00:00:00 EST,1.079914,0.961753,0.532727,0.283709,0.942028,0.161883,0.706796,0.289251,-0.088391,0.788493,0.951385,1.185279,0.569407,0.75645,0.360533,0.167953,0.686686,-0.208681,1.049408,0.71915,0.66114
2019-01-17 00:00:00 EST,0.242863,-0.924427,-0.104237,0.307286,0.416237,-1.322001,-1.66013,-1.238865,-2.342368,0.491453,-0.349784,-0.837348,0.551887,-0.841099,-0.450284,3.237407,-0.456085,-0.598191,1.024738,-0.076581,-0.39409
2019-01-22 00:00:00 EST,-1.78629,0.042848,-0.477425,-0.769039,-0.627385,-0.21385,-1.649826,-0.346168,-0.077097,-0.412043,-1.648861,-0.053818,-0.156785,-0.665327,-0.35319,-0.590299,-0.491604,-1.206656,-1.589459,-1.043991,-1.044731


In [173]:
# $\sigma^2_t = \omega + \sum_{i=1}^p \alpha_i \epsilon^2_{t-i} + \sum^q_{j=1} \beta_j \sigma^2_{t-j}$

garch_models = {}
garch_data = {}

df_transformed_pandas = df_transformed.drop_nulls(subset="log_returns").to_pandas()

for symbol, data in df_transformed_pandas.groupby("symbol"):
    if symbol == "XLE":
        continue

    # 1. Get simple Returns (scaled by 100 often helps convergence)
    # We don't need pre-calculated volatility; GARCH finds it for us.
    log_returns = data['log_returns'] * 100

    # 2. Fit GARCH(1,1)
    # vol='Garch', p=1 (alpha lag), q=1 (beta lag)
    am = arch_model(log_returns, vol='Garch', p=1, q=1)
    res = am.fit(disp='off')

    garch_models[symbol] = res

    # To get the conditional volatility for your strategy:
    cond_vol = res.conditional_volatility
    resid = res.resid
    std_resid = res.std_resid

    data["cond_vol"] = cond_vol
    data["resid"] = resid
    data["std_resid"] = std_resid
    garch_data[symbol] = data

df_garch = pd.concat(garch_data)
df_garch = df_garch.sort_values(by=["timestamp", "symbol"]).reset_index().drop(["level_0", "level_1"], axis=1)
df_garch.head(10)

,symbol,timestamp,Open,High,Low,Close,Volume,trade_count,VWAP,log_returns,volatility,cond_vol,resid,std_resid
0,BKR,2019-01-07 00:00:00-05:00,17.95,18.63,17.86,18.46,3945225.0,31266.0,18.411853,0.028833,NaN,1.907749,2.799546,1.467460
1,COP,2019-01-07 00:00:00-05:00,50.32,50.49,49.25,49.88,5593103.0,52415.0,49.996752,0.012947,NaN,1.843361,1.251268,0.678797
2,CTRA,2019-01-07 00:00:00-05:00,17.38,18.16,17.11,17.86,8704615.0,60948.0,17.914162,0.013348,NaN,1.454035,1.328657,0.913772
3,CVX,2019-01-07 00:00:00-05:00,81.97,83.03,80.81,82.55,4691504.0,50358.0,82.402766,-0.004371,NaN,1.244897,-0.496024,-0.398445
4,DVN,2019-01-07 00:00:00-05:00,17.77,18.51,17.50,18.23,6631284.0,48309.0,18.202379,0.007650,NaN,2.091192,0.700057,0.334764
5,EOG,2019-01-07 00:00:00-05:00,71.11,72.56,69.40,71.15,3933068.0,44612.0,71.514462,0.010347,NaN,1.780483,0.975169,0.547699
6,EQT,2019-01-07 00:00:00-05:00,17.75,19.12,17.64,18.31,5975530.0,39713.0,18.513026,0.030126,NaN,2.989959,2.962377,0.990775
7,EXE,2019-01-07 00:00:00-05:00,370.58,391.44,360.96,380.21,181880.0,54959.0,381.320660,0.031150,NaN,4.234588,3.038790,0.717612
8,FANG,2019-01-07 00:00:00-05:00,78.27,80.31,77.29,79.55,1963001.0,24764.0,79.251615,0.021637,NaN,1.936870,1.998710,1.031928
9,HAL,2019-01-07 00:00:00-05:00,25.05,25.24,24.36,24.98,9148689.0,57164.0,24.989643,0.013124,NaN,1.931571,1.178934,0.610350


In [ ]:
from datetime import timedelta


r_matrix = z_matrix.rolling(index_column="timestamp", period="1d").agg(lambda x: np.corrcoef(x))
r_matrix

TypeError: cannot create expression literal for value of type function.

Hint: Pass `allow_object=True` to accept any value and create a literal of type Object.

In [111]:
# Calculate the Rolling Correlation Matrix (R)
# The paper suggests a lookback (e.g., 60 days or 200 days)
lookback_period = 60
R_matrix = Z_matrix.rolling(window=lookback_period).corr()
R_matrix.tail(50)
# R_matrix is now a MultiIndex DataFrame (Date, Symbol) -> Correlation to other Symbols

symbol                                 BKR       COP      CTRA       CVX  \
timestamp                 symbol                                           
2025-12-15 00:00:00-05:00 OXY     0.545172  0.733329  0.606095  0.551754   
                          PSX     0.440437  0.676266  0.462726  0.685705   
                          SLB     0.524888  0.717907  0.458238  0.517308   
                          TPL     0.263331  0.282643  0.361713  0.313950   
                          TRGP    0.440970  0.571872  0.608768  0.503816   
                          VLO     0.328120  0.386113  0.358549  0.426127   
                          WMB     0.360523  0.209878  0.143218  0.250162   
                          XOM     0.449332  0.757452  0.584566  0.818224   
2025-12-16 00:00:00-05:00 BKR     1.000000  0.508898  0.343894  0.467510   
                          COP     0.508898  1.000000  0.596144  0.753806   
                          CTRA    0.343894  0.596144  1.000000  0.620900   
                          CVX     0.467510  0.753806  0.620900  1.000000   
                          DVN     0.470659  0.858576  0.648061  0.679941   
                          EOG     0.478046  0.839664  0.591003  0.764186   
                          EQT     0.387850  0.456734  0.619552  0.419795   
                          EXE     0.418008  0.507396  0.679271  0.480117   
                          FANG    0.502627  0.809536  0.606800  0.742603   
                          HAL     0.544281  0.600649  0.422674  0.484722   
                          KMI     0.238507  0.206481  0.311092  0.335919   
                          MPC     0.544764  0.413862  0.207099  0.399841   
                          OKE     0.480469  0.546456  0.513475  0.518781   
                          OXY     0.536536  0.751121  0.612854  0.571373   
                          PSX     0.442789  0.657164  0.457658  0.677977   
                          SLB     0.524794  0.730625  0.481964  0.560619   
                          TPL     0.264123  0.337249  0.378996  0.343865   
                          TRGP    0.439121  0.591555  0.623610  0.536188   
                          VLO     0.333622  0.370632  0.347722  0.405140   
                          WMB     0.360601  0.220933  0.159391  0.280741   
                          XOM     0.452142  0.769119  0.591607  0.824913   
2025-12-17 00:00:00-05:00 BKR     1.000000  0.528886  0.347792  0.475476   
                          COP     0.528886  1.000000  0.582664  0.737452   
                          CTRA    0.347792  0.582664  1.000000  0.608038   
                          CVX     0.475476  0.737452  0.608038  1.000000   
                          DVN     0.477797  0.858939  0.640146  0.666947   
                          EOG     0.495152  0.823087  0.577350  0.748633   
                          EQT     0.388659  0.460444  0.618794  0.416928   
                          EXE     0.420182  0.517101  0.681206  0.481257   
                          FANG    0.515410  0.781299  0.590051  0.713578   
                          HAL     0.549358  0.576312  0.402240  0.458584   
                          KMI     0.246186  0.230242  0.324916  0.353934   
                          MPC     0.548937  0.443122  0.223023  0.415053   
                          OKE     0.484052  0.560652  0.515683  0.522500   
                          OXY     0.542657  0.745975  0.603475  0.555276   
                          PSX     0.447007  0.651813  0.447151  0.669736   
                          SLB     0.525821  0.740959  0.475383  0.555844   
                          TPL     0.252028  0.292179  0.349038  0.308533   
                          TRGP    0.443064  0.582432  0.616357  0.523012   
                          VLO     0.338517  0.410315  0.364080  0.430979   
                          WMB     0.365170  0.255900  0.174765  0.305566   
                          XOM     0.457029  0.752675  0.575325  0.814109   

symbol                                 DVN     

### Rationale behind this...


In [127]:
def calculate_conditional_expectation(current_Z, current_R):
    """
    current_Z: Vector of standardized innovations for today (1D array)
    current_R: Correlation matrix for today (2D array)
    """
    signals = {}
    tickers = current_R.columns

    # We iterate through each stock to predict it using its peers
    for _, target_ticker in enumerate(tickers):
        # 1. Isolate the target stock (i) vs the Peers (J)
        # We want to predict 'target_ticker' using 'peers'

        # Get correlations of Target vs Peers (Row i, excluding Col i)
        # R_iJ shape: (1, N-1)
        r_iJ = current_R.loc[target_ticker].drop(target_ticker).values

        # Get Correlation matrix of Peers (Everything excluding i)
        # R_JJ shape: (N-1, N-1)
        r_JJ = current_R.drop(index=target_ticker, columns=target_ticker).values

        # Get actual Z-scores of Peers
        # z_J shape: (N-1, )
        z_J = current_Z.drop(target_ticker).values

        # 2. Solve for Expected Z (The Math: Mean_cond = R_iJ * inv(R_JJ) * Z_J)
        # Using linalg.solve is faster/stabler than actually inverting the matrix
        # Equivalent to: weights = r_iJ @ np.linalg.inv(r_JJ)
        try:
            # Correct "Solver" approach:
            # We need R_JJ^-1 * Z_J.
            # Let x = R_JJ^-1 * Z_J  =>  R_JJ * x = Z_J
            expected_z = r_iJ @ np.linalg.pinv(r_JJ) @ z_J
            # x = np.linalg.solve(r_JJ, z_J)
            # expected_z = np.dot(r_iJ, x)

        except np.linalg.LinAlgError:
            expected_z = 0.0 # Fallback if matrix is singular

        signals[target_ticker] = expected_z

    return signals

# Most recent date to test this on
recent_z = Z_matrix.loc["2025-12-17 00:00:00-05:00"]
recent_r = R_matrix.loc["2025-12-17 00:00:00-05:00"]

pred_signals = pd.Series(calculate_conditional_expectation(recent_z, recent_r))
signal = recent_z - pred_signals
signal

symbol
BKR     0.239978
COP    -0.059546
CTRA   -0.178488
CVX     0.103710
DVN     0.216658
EOG    -0.349366
EQT     0.280079
EXE    -0.189489
FANG   -0.572251
HAL     0.275473
KMI    -0.470258
MPC    -0.608397
OKE     0.224673
OXY    -0.138986
PSX     0.055661
SLB     0.187522
TPL     0.706890
TRGP   -0.241780
VLO     0.880639
WMB     0.507305
XOM     0.545729
dtype: float64

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

def get_clean_residuals(price_df, sector_ticker="XLE", lookback=60):
    """
    1. Calculates Log Returns.
    2. Regresses each stock against the Sector (XLE).
    3. Returns the 'Clean Residuals' to be used for GARCH.
    """
    # 1. Calculate Log Returns
    log_rets = np.log(price_df / price_df.shift(1)).dropna()

    # Separate Sector vs Stocks
    sector_rets = log_rets[sector_ticker].values.reshape(-1, 1)
    stocks_rets = log_rets.drop(columns=[sector_ticker])

    # Store results
    clean_residuals_df = pd.DataFrame(index=stocks_rets.index, columns=stocks_rets.columns)
    betas = {}

    # 2. Rolling Regression (Inefficient loop, but clear logic)
    # Note: For production, you might want to use a rolling window approach.
    # Here we simulate the 'latest' residual calculation.

    # Iterate through time to prevent lookahead bias (Rolling Window)
    # Start from 'lookback' index
    for t in range(lookback, len(log_rets)):

        # Define Window
        window_start = t - lookback
        window_end = t

        # Get Window Data
        y_window = stocks_rets.iloc[window_start:window_end]
        X_window = sector_rets[window_start:window_end]

        # Get Current Data (for the residual of 'today')
        y_today = stocks_rets.iloc[t]
        X_today = sector_rets[t].reshape(1, -1)

        # Iterate through each stock to regress
        for symbol in stocks_rets.columns:
            y_col = y_window[symbol].values

            # Simple OLS: R_stock = alpha + beta * R_sector + epsilon
            model = LinearRegression().fit(X_window, y_col)

            # Predict what the stock 'should' have done today based on XLE
            expected_ret = model.predict(X_today)[0]
            actual_ret = y_today[symbol]

            # The Residual: This is the "Clean" return
            clean_residuals_df.loc[log_rets.index[t], symbol] = actual_ret - expected_ret

    return clean_residuals_df.dropna()

# --- HOW TO USE THIS IN YOUR PIPELINE ---

# 1. Get the Clean Residuals
# df_prices should contain columns ['XLE', 'XOM', 'CVX', ...]
df_clean_residuals = get_clean_residuals(df_prices, sector_ticker="XLE", lookback=60)

# 2. Run GARCH on THESE residuals (Not on raw prices!)
# This ensures your volatility measure is "Idiosyncratic Volatility"

garch_results = {}
final_z_scores = pd.DataFrame()

for symbol in df_clean_residuals.columns:
    # Scale by 100 for GARCH convergence
    resid_series = df_clean_residuals[symbol].astype(float) * 100

    # Fit GARCH
    am = arch_model(resid_series, vol='Garch', p=1, q=1)
    res = am.fit(disp='off')

    # Get Volatility
    cond_vol = res.conditional_volatility

    # 3. Calculate "Clean" Standardized Innovations (Z-Scores)
    # Z = Clean_Residual / Idiosyncratic_Volatility
    z_score = resid_series / cond_vol
    final_z_scores[symbol] = z_score

# 4. NOW calculate your R Matrix on these clean Z-scores
# This matrix will be "free of systematic patterns"
R_matrix = final_z_scores.rolling(window=60).corr()